In [1]:
import sys
import numpy as np
import dill
import matplotlib.pyplot as plt
%matplotlib inline

import coremagmodels as cm

sys.path.append('../')
import coreflows as cf
sf = cf.advect.SteadyFlow()
wv = cf.advect.Waves()

# Create Synthetic Data to Analyze

## Create background magnetic field

In [2]:
plot_figures = False

In [ ]:
magmod = cm.models.Chaos6()
T_start = 2001
T_end = 2015
l_max = 18
Nth = 18*2+2
th, ph = magmod.get_thvec_phvec_DH(l_max=l_max)
Nt = (T_end-T_start)*3
T = np.linspace(T_start, T_end, Nt)

In [8]:
# Import steady flows
filename = '../coreflows/data/steady_flow_fortran_fit'
th_sf,ph_sf,vth_sf,vph_sf = sf.import_fortran_flow_DH(filename)
vth_sfSH = sf.v2vSH(vth_sf)
vph_sfSH = sf.v2vSH(vph_sf)

In [10]:
# Import and create analytic waves 
data = dill.load(open('/Users/nknezek/code/coreflows/coreflows/data/wavefits012.p','rb'))
c012 = data['c012']
f012 = data['f012']

In [11]:
# compute magnetic field data over timeframe
B_lmax = 18

th, ph = magmod.get_thvec_phvec_DH(Nth)

Bsh = magmod.get_sht_allT(T, l_max = B_lmax)
B = magmod.B_sht_allT(Bsh, Nth=Nth, l_max=B_lmax)
_, dthB, dphB = magmod.gradB_sht_allT(Bsh, Nth=Nth, l_max=B_lmax)

SVsh = magmod.get_SVsht_allT(T, l_max=B_lmax)
SV = magmod.B_sht_allT(SVsh, Nth=Nth, l_max=B_lmax)
_, dthSV, dphSV = magmod.gradB_sht_allT(SVsh, Nth=Nth, l_max=B_lmax)

SAsh = magmod.get_SAsht_allT(T, l_max=B_lmax)
SA = magmod.B_sht_allT(SAsh, Nth=Nth, l_max=B_lmax)
_, dthSA, dphSA = magmod.gradB_sht_allT(SAsh, Nth=Nth, l_max=B_lmax)

if plot_figures:
    i = 0
    plt.figure(figsize=(16,8))
    plt.subplot(131)
    cm.plot.contourf_DH(B[i,:,:]/1e6, newfig=False, title='B {}'.format(T[i]), clbl='mT',zmax=1.2)
    plt.subplot(132)
    cm.plot.contourf_DH(SV[i,:,:]/1e3, newfig=False, title='SV {}'.format(T[i]), clbl='uT/yr',zmax=25)
    plt.subplot(133)
    cm.plot.contourf_DH(SA[i,:,:]/1e3, newfig=False, title='SA {}'.format(T[i]), clbl='uT/yr^2',zmax=2.)

In [12]:
# compute magnetic field data over timeframe
Bd_lmax = 1

th, ph = magmod.get_thvec_phvec_DH(Nth)

Bdsh = magmod.get_sht_allT(T, l_max = Bd_lmax)
Bd = magmod.B_sht_allT(Bdsh, Nth=Nth, l_max=Bd_lmax)
_, dthBd, dphBd = magmod.gradB_sht_allT(Bdsh, Nth=Nth, l_max=Bd_lmax)

SVdsh = magmod.get_SVsht_allT(T, l_max=Bd_lmax)
SVd = magmod.B_sht_allT(SVdsh, Nth=Nth, l_max=Bd_lmax)
_, dthSVd, dphSVd = magmod.gradB_sht_allT(SVdsh, Nth=Nth, l_max=Bd_lmax)

SAdsh = magmod.get_SAsht_allT(T, l_max=Bd_lmax)
SAd = magmod.B_sht_allT(SAdsh, Nth=Nth, l_max=Bd_lmax)
_, dthSAd, dphSAd = magmod.gradB_sht_allT(SAdsh, Nth=Nth, l_max=Bd_lmax)

if plot_figures:
    i = 0
    plt.figure(figsize=(16,8))
    plt.subplot(131)
    cm.plot.contourf_DH(Bd[i,:,:]/1e6, newfig=False, title='Bd {}'.format(T[i]), clbl='mT')
    plt.subplot(132)
    cm.plot.contourf_DH(SVd[i,:,:]/1e3, newfig=False, title='SVd {}'.format(T[i]), clbl='uT/yr')
    plt.subplot(133)
    cm.plot.contourf_DH(SAd[i,:,:]/1e3, newfig=False, title='SAd {}'.format(T[i]), clbl='uT/yr^2')

## SV and SA from Steady Flow

In [25]:
steadyflow_lmax = 14
SV_steadyflow = sf.SV_steadyflow_allT(vth_sfSH, vph_sfSH, Bsh, magmodel=magmod, Nth=Nth, B_lmax=B_lmax, v_lmax=steadyflow_lmax)
SV_resid = SV-SV_steadyflow

SA_steadyflow = sf.SA_steadyflow_allT(vth_sfSH, vph_sfSH, SVsh, magmodel=magmod, Nth=Nth, B_lmax=B_lmax, v_lmax=steadyflow_lmax)
SA_resid = SA- SA_steadyflow
if plot_figures:
    cm.plot.contourf_DH(SV_steadyflow[0,:,:]/1e3, title='SV from steady flow', clbl='uT/yr')
    cm.plot.contourf_DH(SA_steadyflow[0,:,:]/1e3, title='SA from steady flow', clbl='uT/yr^2')

## Simulated Wave SA and SV

In [26]:
l_w1 = 0
m_w1 = 6
delta_th_w1 = 8
phase_w1 = 120
period_w1 = 7.5
vmax_w1 = 1.5 #km/yr
dat_w1 = (l_w1, m_w1, period_w1, vmax_w1, phase_w1, delta_th_w1)

vw1th, vw1ph, aw1th, aw1ph = wv.vel_accel_allT(dat_w1, T, c012, Nth)
divvw1 = wv.div_allT(vw1th, vw1ph)
divaw1 = wv.div_allT(aw1th, aw1ph)
SVw1 = wv.SV_wave_allT(B, dthB, dphB, vw1th, vw1ph, divvw1)
SAw1 = wv.SA_wave_fluidaccel_allT(B, dthB, dphB, aw1th, aw1ph, divaw1)
SAw1 += wv.SA_wave_magSV_allT(SV, dthSV, dphSV, vw1th, vw1ph, divvw1)
SAdw1 = wv.SA_wave_fluidaccel_allT(Bd, dthBd, dphBd, aw1th, aw1ph, divaw1)
SVdw1 = wv.SV_wave_allT(Bd, dthBd, dphBd, vw1th, vw1ph, divvw1)

if plot_figures:
    plt.figure(figsize=(16,14))
    i = -1
    qarr_scale_mod = 1e-6
    dq = 2
    plt.subplot(321)
    cm.plot.contourf_DH(divvw1[i,:,:], clbl='km/yr', newfig=False)
    cm.plot.base_quiver(vw1th[i,::dq,::dq], vw1ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Velocity from wave 1',)
    plt.subplot(322)
    cm.plot.contourf_DH(divaw1[i,:,:], clbl='km/yr^2', newfig=False)
    cm.plot.base_quiver(aw1th[i,::dq,::dq], aw1ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Acceleration Wave 1', )

    plt.subplot(323)
    cm.plot.contourf_DH(SVw1[i,:,:]/1e3, title='secular variation from wave 1', clbl='uT/yr', newfig=False)
    plt.subplot(324)
    cm.plot.contourf_DH(SAw1[i,:,:]/1e3, title='Secular Acceleration Wave 1', clbl='uT/yr^2', newfig=False)

    plt.subplot(325)
    cm.plot.contourf_DH(SVdw1[i,:,:]/1e3, title='secular variation from wave 1, Dipole', clbl='uT/yr', newfig=False)
    plt.subplot(326)
    cm.plot.contourf_DH(SAdw1[i,:,:]/1e3, title='Secular Acceleration Wave 1, Dipole', clbl='uT/yr^2', newfig=False)
    plt.tight_layout()

In [27]:
l_w2 = 0
m_w2 = 3
delta_th_w2 = 14
phase_w2 = 220
period_w2 = 10.5
vmax_w2 = 2
dat_w2 = (l_w2, m_w2, period_w2, vmax_w2, phase_w2, delta_th_w2)

vw2th, vw2ph, aw2th, aw2ph = wv.vel_accel_allT(dat_w2, T, c012, Nth)
divvw2 = wv.div_allT(vw2th, vw2ph)
divaw2 = wv.div_allT(aw2th, aw2ph)
SVw2 = wv.SV_wave_allT(B, dthB, dphB, vw2th, vw2ph, divvw2)
SAw2 = wv.SA_wave_fluidaccel_allT(B, dthB, dphB, aw2th, aw2ph, divaw2)
SAw2 += wv.SA_wave_magSV_allT(SV, dthSV, dphSV, vw2th, vw2ph, divvw2)
SAdw2 = wv.SA_wave_fluidaccel_allT(Bd, dthBd, dphBd, aw2th, aw2ph, divaw2)
SVdw2 = wv.SV_wave_allT(Bd, dthBd, dphBd, vw2th, vw2ph, divvw2)

if plot_figures:
    plt.figure(figsize=(16,14))
    i = -1
    dq = 2
    qarr_scale_mod = 1e-6
    plt.subplot(321)
    cm.plot.contourf_DH(divvw2[i,:,:], clbl='km/yr', newfig=False)
    cm.plot.base_quiver(vw2th[i,::dq,::dq], vw2ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Velocity from Wave 2')
    plt.subplot(322)
    cm.plot.contourf_DH(divaw2[i,:,:], clbl='km/yr^2', newfig=False)
    cm.plot.base_quiver(aw2th[i,::dq,::dq], aw2ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Acceleration Wave 2')

    plt.subplot(323)
    cm.plot.contourf_DH(SVw2[i,:,:]/1e3, title='secular variation from Wave 2', clbl='uT/yr', newfig=False)
    plt.subplot(324)
    cm.plot.contourf_DH(SAw2[i,:,:]/1e3, title='Secular Acceleration Wave 2', clbl='uT/yr^2', newfig=False)
    plt.tight_layout()

    plt.subplot(325)
    cm.plot.contourf_DH(SVdw2[i,:,:]/1e3, title='secular variation from Wave 2, Dipole', clbl='uT/yr', newfig=False)
    plt.subplot(326)
    cm.plot.contourf_DH(SAdw2[i,:,:]/1e3, title='Secular Acceleration Wave 2, Dipole', clbl='uT/yr^2', newfig=False)
    plt.tight_layout()

In [28]:
l_w3 = 1
m_w3 = 6
delta_th_w3 = 8
phase_w3 = 50
period_w3 = 8.5
vmax_w3 = 2
dat_w3 = (l_w3, m_w3, period_w3, vmax_w3, phase_w3, delta_th_w3)

vw3th, vw3ph, aw3th, aw3ph = wv.vel_accel_allT(dat_w3, T, c012, Nth)
divvw3 = wv.div_allT(vw3th, vw3ph)
divaw3 = wv.div_allT(aw3th, aw3ph)
SVw3 = wv.SV_wave_allT(B, dthB, dphB, vw3th, vw3ph, divvw3)
SAw3 = wv.SA_wave_fluidaccel_allT(B, dthB, dphB, aw3th, aw3ph, divaw3)
SAw3 += wv.SA_wave_magSV_allT(SV, dthSV, dphSV, vw3th, vw3ph, divvw3)
SAdw3 = wv.SA_wave_fluidaccel_allT(Bd, dthBd, dphBd, aw3th, aw3ph, divaw3)
SVdw3 = wv.SV_wave_allT(Bd, dthBd, dphBd, vw3th, vw3ph, divvw3)

if plot_figures:
    plt.figure(figsize=(16,14))
    i = -1
    dq = 2
    qarr_scale_mod = 1e-6
    plt.subplot(321)
    cm.plot.contourf_DH(divvw3[i,:,:], clbl='km/yr', newfig=False)
    cm.plot.base_quiver(vw3th[i,::dq,::dq], vw3ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Velocity from Wave 2')
    plt.subplot(322)
    cm.plot.contourf_DH(divaw3[i,:,:], clbl='km/yr^2', newfig=False)
    cm.plot.base_quiver(aw3th[i,::dq,::dq], aw3ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Acceleration Wave 2')

    plt.subplot(323)
    cm.plot.contourf_DH(SVw3[i,:,:]/1e3, title='secular variation from Wave 2', clbl='uT/yr', newfig=False)
    plt.subplot(324)
    cm.plot.contourf_DH(SAw3[i,:,:]/1e3, title='Secular Acceleration Wave 2', clbl='uT/yr^2', newfig=False)
    plt.tight_layout()

    plt.subplot(325)
    cm.plot.contourf_DH(SVdw3[i,:,:]/1e3, title='secular variation from Wave 2, Dipole', clbl='uT/yr', newfig=False)
    plt.subplot(326)
    cm.plot.contourf_DH(SAdw3[i,:,:]/1e3, title='Secular Acceleration Wave 2, Dipole', clbl='uT/yr^2', newfig=False)
    plt.tight_layout()

In [29]:
l_w4 = 1
m_w4 = -3
delta_th_w4 = 18
phase_w4 = 350
period_w4 = 5.5
vmax_w4 = 2.5
dat_w4 = (l_w4, m_w4, period_w4, vmax_w4, phase_w4, delta_th_w4)

vw4th, vw4ph, aw4th, aw4ph = wv.vel_accel_allT(dat_w4, T, c012, Nth)
divvw4 = wv.div_allT(vw4th, vw4ph)
divaw4 = wv.div_allT(aw4th, aw4ph)
SVw4 = wv.SV_wave_allT(B, dthB, dphB, vw4th, vw4ph, divvw4)
SAw4 = wv.SA_wave_fluidaccel_allT(B, dthB, dphB, aw4th, aw4ph, divaw4)
SAw4 += wv.SA_wave_magSV_allT(SV, dthSV, dphSV, vw4th, vw4ph, divvw4)
SAdw4 = wv.SA_wave_fluidaccel_allT(Bd, dthBd, dphBd, aw4th, aw4ph, divaw4)
SVdw4 = wv.SV_wave_allT(Bd, dthBd, dphBd, vw4th, vw4ph, divvw4)

if plot_figures:
    plt.figure(figsize=(16,14))
    i = -1
    dq = 2
    qarr_scale_mod = 1e-6
    plt.subplot(321)
    cm.plot.contourf_DH(divvw4[i,:,:], clbl='km/yr', newfig=False)
    cm.plot.base_quiver(vw4th[i,::dq,::dq], vw4ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Velocity from Wave 2')
    plt.subplot(322)
    cm.plot.contourf_DH(divaw4[i,:,:], clbl='km/yr^2', newfig=False)
    cm.plot.base_quiver(aw4th[i,::dq,::dq], aw4ph[i,::dq,::dq], newfig=False, qarr_scale_mod=qarr_scale_mod, qkey=1., title='Fluid Acceleration Wave 2')

    plt.subplot(323)
    cm.plot.contourf_DH(SVw4[i,:,:]/1e3, title='secular variation from Wave 2', clbl='uT/yr', newfig=False)
    plt.subplot(324)
    cm.plot.contourf_DH(SAw4[i,:,:]/1e3, title='Secular Acceleration Wave 2', clbl='uT/yr^2', newfig=False)
    plt.tight_layout()

    plt.subplot(325)
    cm.plot.contourf_DH(SVdw4[i,:,:]/1e3, title='secular variation from Wave 2, Dipole', clbl='uT/yr', newfig=False)
    plt.subplot(326)
    cm.plot.contourf_DH(SAdw4[i,:,:]/1e3, title='Secular Acceleration Wave 2, Dipole', clbl='uT/yr^2', newfig=False)
    plt.tight_layout()

In [30]:
# all four waves in one dataset:
SAw = SAw1 + SAw2 + SAw3 + SAw4
SVw = SVw1 + SVw2 + SVw3 + SVw4
SAdw = SAdw1 + SAdw2 + SAdw3 + SAdw4
SVdw = SVdw1 + SVdw2 + SVdw3 + SVdw4

In [34]:
# Store the data
data = {'T_start' : T_start 
        ,'T_end' : T_end
        ,'T' : T
        ,'l_max' : l_max
        ,'Nth' : Nth
        ,'th' : th
        ,'ph' : ph
        ,'Nt' : Nt
        ,'B' : B
        ,'Bd' : Bd
        ,'Bsh' : Bsh
        ,'Bdsh' : Bdsh
        ,'SV' : SV
        ,'SVd' : SVd
        ,'SVsh' : SVsh
        ,'SVdsh' : SVdsh
        ,'dthB' : dthB
        ,'dphB' : dphB
        ,'dthBd' : dthBd
        ,'dphBd' : dphBd
        ,'dthSV' : dthSV
        ,'dphSV' : dphSV
        ,'dthSVd' : dthSVd
        ,'dphSVd' : dphSVd
        ,'SAw1' : SAw1
        ,'SAw2' : SAw2
        ,'SAw3' : SAw3
        ,'SAw4' : SAw4
        ,'SAw' : SAw
        ,'SVw1' : SVw1
        ,'SVw2' : SVw2
        ,'SVw3' : SVw3
        ,'SVw4' : SVw4
        ,'SVw' : SVw
        ,'SA_steadyflow' : SA_steadyflow
        ,'SV_steadyflow' : SV_steadyflow
        ,'dat_w1' : dat_w1
        ,'dat_w2' : dat_w2
        ,'dat_w3' : dat_w3
        ,'dat_w4' : dat_w4
        ,'wave_params': (dat_w1, dat_w2, dat_w3, dat_w4)
        ,'SAdw1' : SAdw1
        ,'SAdw2' : SAdw2
        ,'SAdw3' : SAdw3
        ,'SAdw4' : SAdw4
        ,'SAdw' : SAdw
        ,'SVdw1' : SVdw1
        ,'SVdw2' : SVdw2
        ,'SVdw3' : SVdw3
        ,'SVdw4' : SVdw4
        ,'SVdw' : SVdw}

dill.dump(data, open('data_4_waves.m','wb'))